# Data Pre-Processing

1. Present where we can find the data
2. Discuss the different variables we can use for this project
3. Process the data for the next part
4. Create the different data set  

## I) Where can we find the data

The dataset can be find on the website of Oqali

## II) The features

- The data from the Web-Scraping

**Nom**: The name of the article, examples : "VITTEL Eau minérale naturelle plate 50cl", "SENSEO Dosettes de café classique 54 dosettes 375g"

**Conservation**: How the product can be conservated, examples : "Consommez les dosettes de café SENSEO de préférence dans les 2 à 3 semaines après l'ouverture et conservez-les dans une boîte fermée dans un endroit frais"

**Valeurs nutritionelles**: 

**Ingrédients** : The composition of the article

**Dénomination légale de vente**: 

- The data from Oqali :

**Nom du produit**:

**Dénomination de vente**:

**Mode de conservation**:

**Valeurs nutritionnelles**:

**Ingrédients**: 


## II) NLP Data Processing

### 1) Ingredients

This part can be challenging, this first thing is that they are a lot of different words and some can be useless for our machine learning algortihm. The other thing is that with the data imported from the web scraping the words can be slightly different, so they won't match with our data set.

For the first step, we import the data and keep only the features of interest.

In [14]:
import pandas as pd
import numpy as np
df = pd.read_csv(r'C:\Users\Thomas Aujoux\Documents\GitHub\food-classification\data_preparation\ingredients.csv', dtype={'Code produit': int, 'Secteur': str, "Famille": str, "Ingrédient": str}, sep=';')
df = df[["Code produit", "Secteur", "Famille", "Ingrédient"]]
df = df.rename(columns={ 'Code produit': 'Code_produit' })


C:\Users\Thomas Aujoux\AppData\Local\Temp\ipykernel_15816\3188806925.py:3: DtypeWarning: Columns (4,5,6,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\Thomas Aujoux\Documents\GitHub\food-classification\data_preparation\ingredients.csv', dtype={'Code produit': int, 'Secteur': str, "Famille": str, "Ingrédient": str}, sep=';')


In [15]:
df.head()

,Code_produit,Secteur,Famille,Ingrédient
0,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,lait ecreme reconstitue
1,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,sucre
2,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,fruit : framboise
3,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,creme
4,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,"epaississants : amidon transforme de mais, gom..."


We can see on the DataFrame that they are 4 features.
The feature called Ingrédient contains too many informations, we want to have a clean "Ingrédient" without too many possibilities in order to match easily with the data from the web-scraping.
1. We will delete every ingredients after the ":" which describes us the composition of this ingredient. We can give one exemple with "chocolat noir : masse de cacao, sucre, emulsifiant : lecithine de soja"
We will do this for 3 reasons, the first one is that some data from the web-scraping won't put every detail about the ingredient but just "chocolat noir" because the data from Oqali are really precise. The second is that they are not the main composition of the product, they are just a part of an ingredient, we can suppose that in the majority their % in the product is really low. Finally we tested different classification with them and without them and the result where almost the same.
+ add that they will introduce the unicity of the ingredient
+ they are "conservateur", "epaississant", ect ... not really important and they composition can vary from a brand to another
2. We will delete every ingredients that are in "()" for the same reasons as ":"
3. To see if the data is clean and for the rest of the study we will remplace every " " with "_"
4. We drop every rows that contain an ingredient which is unique in the data

In [16]:
df = df[~df['Secteur'].isin(['Sauces condimentaires_RHF', 'Bouillons et potages_RHF'])]
df.Ingrédient[df.Ingrédient.str.contains(":")] = df[df.Ingrédient.str.contains(":")].Ingrédient.str.split().map(lambda x: x[0])
df.Ingrédient[df.Ingrédient.str.contains("\(")] = df[df.Ingrédient.str.contains("\(")].Ingrédient.str.replace(r"\s\(.*\)", "", regex=True)
df.Ingrédient[df.Ingrédient.str.contains("\(")] = df[df.Ingrédient.str.contains("\(")].Ingrédient.str.replace(r"\s\(.*\)\s", " ", regex=True)
df.Ingrédient[df.Ingrédient.str.contains("\(")] = df[df.Ingrédient.str.contains("\(")].Ingrédient.str.replace(r"\(.*\)", "", regex=True)
df.Ingrédient[df.Ingrédient.str.contains("\(")] = df[df.Ingrédient.str.contains("\(")].Ingrédient.str.replace(r"\(.*", "", regex=True)
df.Ingrédient = df["Ingrédient"].apply(lambda x: x.rstrip())
df.Ingrédient = df["Ingrédient"].apply(lambda x: x.lstrip())
df = df[df.duplicated(subset=['Ingrédient'], keep=False)]
df_ = df.copy()
df_['Ingrédient'] = df_['Ingrédient'].apply(lambda x : x.replace(' ','_'))

C:\Users\Thomas Aujoux\AppData\Local\Temp\ipykernel_15816\3092734722.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Ingrédient[df.Ingrédient.str.contains("\(")] = df[df.Ingrédient.str.contains("\(")].Ingrédient.str.replace(r"\s\(.*\)", "", regex=True)
C:\Users\Thomas Aujoux\AppData\Local\Temp\ipykernel_15816\3092734722.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Ingrédient[df.Ingrédient.str.contains("\(")] = df[df.Ingrédient.str.contains("\(")].Ingrédient.str.replace(r"\s\(.*\)\s", " ", regex=True)
C:\Users\Thomas Aujoux\AppData\Local\Temp\ipykernel_15816\3092734722.py:5: SettingWithCopyWarning: 
A v

In [17]:
df = df.drop(index=[1167903,
                    1053668,
                    875941,
                    #1158191,
                    #382675,
                    701702,
                    1202560,
                    1412604,
                    489683,
                    1441080,
                    1441208,
                    20633,
                    429562
                    ])
df_ = df_.drop(index=[1167903,
                    1053668,
                    875941,
                    #1158191,
                    #382675,
                    701702,
                    1202560,
                    1412604,
                    489683,
                    1441080,
                    1441208,
                    20633,
                    429562
                    ])

In [19]:
df.head()

,Code_produit,Secteur,Famille,Ingrédient
0,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,lait ecreme reconstitue
1,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,sucre
2,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,fruit
3,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,creme
4,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,epaississants


In [21]:
df_.head()

,Code_produit,Secteur,Famille,Ingrédient
0,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,lait_ecreme_reconstitue
1,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,sucre
2,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,fruit
3,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,creme
4,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,epaississants


In [ ]:
a = sorted(df["Code_produit"].unique())
b = sorted(df["Secteur"].unique(), key=str.lower)
c = sorted(df["Famille"].unique(), key=str.lower)
d = sorted(df["Ingrédient"].unique(), key=str.lower)
e = df.Ingrédient[df.Ingrédient.str.contains(":")]
f = df.Ingrédient[df.Ingrédient.str.contains("\(")]
print(a[0:10], b[0:10], c[0:10], d[0:10])

[450, 453, 455, 456, 460, 463, 464, 496, 498, 513] ['Aliments infantiles de diversification', 'Aperitifs a croquer', 'Barres cerealieres', 'Biscuits et gateaux industriels', 'Boissons Rafraichissantes Sans Alcool', 'Bouillons et potages', 'Bouillons et potages_RHF', 'Cereales pour le petit dejeuner', 'Charcuterie', 'Chocolat et produits chocolates'] ['Arachides et graines', 'Assortiment de glaces', 'Assortiments', 'Assortiments de chocolats', 'Assortiments de confiseries', 'Assortiments de mignardises', 'Assortiments de snacking', 'Aumonieres aperitives', 'Autre fromage a pate persillee', 'Autre fromage a pate pressee cuite'] ['3_poivres', 'abondance_aop', 'abricot', 'abricot_de_provence', 'abricot_deshydrate', 'abricot_lyophilise', 'abricot_nectarine', 'abricot_orange', 'abricot_peche', 'abricot_sec']


In [13]:
df.to_csv(r'C:\Users\Thomas Aujoux\Documents\GitHub\food-classification\data_preparation\clean.csv')
df_.to_csv(r'C:\Users\Thomas Aujoux\Documents\GitHub\food-classification\data_preparation\clean_.csv')

In [22]:
df_new = df.groupby(['Code_produit', 'Secteur'])['Ingrédient'].agg(lambda col: ' '.join(col)).reset_index(name='Ingrédient')

from sklearn.feature_extraction.text import CountVectorizer
coun_vect = CountVectorizer(token_pattern='\\b(\\w+[\\.,%_1234567890:()]?\\w+)\\b')
count_matrix = coun_vect.fit_transform(df_new['Ingrédient'])
count_array = count_matrix.toarray()
df_vect = pd.DataFrame(data=count_array,columns = coun_vect.get_feature_names_out())
new_df = pd.concat([df_new[["Code_produit", "Secteur"]], df_vect], axis=1)

In [24]:
df_new = df.groupby(['Code_produit', 'Secteur'])['Ingrédient'].agg(lambda col: ' '.join(col)).reset_index(name='Ingrédient')

from sklearn.feature_extraction.text import CountVectorizer
coun_vect = CountVectorizer(token_pattern=r"(?:(?<=^)|(?<=\s))([a-zA-Z0-9]+)(?:(?=\s)|(?=$))")
count_matrix = coun_vect.fit_transform(df_new['Ingrédient'])
count_array = count_matrix.toarray()
df_vect = pd.DataFrame(data=count_array,columns = coun_vect.get_feature_names_out())
new_df2 = pd.concat([df_new[["Code_produit", "Secteur"]], df_vect], axis=1)

In [35]:
new_df.head()

,Code_produit,Secteur,3_poivres,_ananas,"_b1,_b9","_b1,_b9_et_provitamine_a",_b12,_caillettes_de_boeuf_en_proportions_naturelles,"_carotte,_potiron_et_spiruline","_d_avoine,_de_riz_et_d_epeautre",...,zestes_d_orange,zestes_d_oranges,zestes_de_citron,zestes_de_citron_deshydrates,zestes_de_citron_jaune,zestes_de_citron_vert,zestes_de_citrons,zestes_de_mandarine,zinc,zygochlamys_patagonica
0,450,Produits laitiers et desserts frais,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,453,Produits laitiers et desserts frais,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,455,Produits laitiers et desserts frais,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,456,Produits laitiers et desserts frais,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,460,Produits laitiers et desserts frais,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
z = pd.DataFrame(new_df.columns.values)
y = pd.DataFrame(new_df2.columns.values)

In [8]:
u = new_df2.loc[53]

In [9]:
new_df = new_df.drop(['_ananas', '_b1,_b9', '_b1,_b9_et_provitamine_a', '_b12', '_caillettes_de_boeuf_en_proportions_naturelles', '_carotte,_potiron_et_spiruline', '_d_avoine,_de_riz_et_d_epeautre', '_d_orge_et_de_riz', '_de_cassis_et_de_mure', '_de_cuivre', '_de_cuivre_et_de_manganese', '_de_matiere_grasse', '_de_matieres_grasses_au_lait_demi_ecreme_pasteurise', '_de_mg', '_de_thym,_de_laurier', '_e', '_gr', '_lb.rhamnosus', '_maigre_de_tete,_viande_et_couenne_de_porc', '_mg', '_paprika', '_provitamine_a', '_rooibos_et_du_melange_de_13_plantes_ricola', '_tournesol', '_vanille'], axis=1)

In [11]:
new_df.to_csv(r'C:\Users\Thomas Aujoux\Documents\GitHub\food-classification\data_preparation\clean.csv')
new_df2.to_csv(r'C:\Users\Thomas Aujoux\Documents\GitHub\food-classification\data_preparation\clean2.csv')

KeyboardInterrupt: 